In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os
import tensorflow as tf 
import pathlib
import sys
import random
import shutil
import cv2 as cv
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tensorflow.keras import layers
from scipy.spatial import distance 
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors

In [2]:
IMAGE_SIZE = 256
train_data = pathlib.Path('/kaggle/input/.../data/train/')
test_data = pathlib.Path('/kaggle/input/.../data/test/')
output = pathlib.Path('/kaggle/working/')

In [3]:
kernel_size = 3
def HECOLOR (image):
    channels = cv.split(image)
    eq_channels = []
    for ch, color in zip(channels, ['B', 'G', 'R']):
        eq_channels.append(cv.equalizeHist(ch))

    eq_image = cv.merge(eq_channels)
    eq_image = cv.cvtColor(eq_image, cv.COLOR_BGR2RGB)

    return eq_image/255.0

def features (image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     image = cv.equalizeHist(image)
    #Gaussian filter
    image = cv.GaussianBlur(image, (kernel_size, kernel_size), 0)
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(16,16),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    return hog_image

In [4]:
def resize_and_rescale(image):
    image = tf.cast(image, tf.float32)
    image = (image / 255.0)
    return image

def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory)
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    file_names = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names += [os.path.join(label_directory, f)
                       for f in os.listdir(label_directory)]

    random.shuffle(file_names)

    for f in file_names:
        img = cv.imread(f)
        img = features(img)
#         img = np.array(HECOLOR(img))
#         img = resize_and_rescale(img)
        dirname = os.path.split(os.path.dirname(f))[1]

        images.append(img)
        labels.append(int(dirname))       

    images, labels = np.asarray(images), np.asarray(labels)

    return images, labels

In [5]:

imagens, labels = load_data(train_data)
imagens = imagens.reshape(imagens.shape[0],-1)
print(imagens.shape)

imgs_test, labs_test = load_data(test_data)
imgs_test = imgs_test.reshape(imgs_test.shape[0],-1)
print(imgs_test.shape)

print('[INFO]: Classification starting....')
model_NN = NearestNeighbors(n_neighbors=2, algorithm='ball_tree')
model_NN.fit(imagens, labels)

labs_predict = model_NN.predict(imgs_test)

accuracy = metrics.accuracy_score(labs_test, labs_predict)
f1 = metrics.f1_score(labs_test, labs_predict, average="macro")
precision = metrics.precision_score(labs_test, labs_predict, average="macro") 
recall = metrics.recall_score(labs_test, labs_predict, average="macro")
print("The accuracy= {} f1_score= {} precison = {} recall= {}".format(accuracy, f1, precision, recall))
cm = metrics.confusion_matrix(labs_test, labs_predict)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()